In [2]:
import json
import datetime
import pandas as pd
import numpy as np
#import databricks.koalas as ks
file_name = 'logs/20210214185653.log'

pd.read_json(file_name)

ValueError: Expected object or value

In [ ]:
#tweets = [data[key] for key in data]
tweets_df = pd.DataFrame(tweets)
tweets_df['created_at'] = tweets_df['created_at'].apply(lambda date: datetime.datetime.strptime(date[:16], "%Y-%m-%dT%H:%M"))
tweets_df['folder'] = tweets_df['created_at'].dt.strftime('%Y/%m/%d/%H')
tweets_df.head()

In [ ]:
from io import StringIO 
import boto3

def store_tags(row):
    s3_bucket = 'tweet.watcher'
    s3_client = boto3.resource('s3')
    tags_rows = []
    for key in row['tags']:
        for value in row['tags'][key]:
            tags_rows += [(key, value, 1)]
    df = pd.DataFrame(tags_rows, columns=['tag', 'value', 'count'])
    df = df.groupby(['tag', 'value']).count()
    csv_buffer = StringIO()
    df.to_csv(csv_buffer)
    s3_client.Object(s3_bucket, f'{row["folder"]}/{row["id"]}_tags.csv').put(Body=csv_buffer.getvalue())
    
tweets_df[['folder', 'id', 'tags']].apply(store_tags, axis=1)

In [ ]:
dataframes = []
for tag in tag_collection:
    df = pd.DataFrame(tag_collection[tag], columns=['created_at', 'value', 'count'])
    df['tag'] = tag
    for d in df.groupby('created_at'):
        dataframes += [d[1]]

tags_df = pd.concat(dataframes)
tags_df.head()

In [ ]:
counts = tags_df.groupby(['created_at', 'value', 'tag']).count()
counts

In [ ]:
counts.loc[counts['count'] > 1].sort_values('created_at')

In [ ]:
query = None
with open(file_name) as f:
    query = f.load(json_file)